## Create Hive Tables

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import broadcast

In [4]:
spark = SparkSession.builder.appName("Spark features").getOrCreate()

### Remove existing directories if exists

In [57]:
from shutil import rmtree
import os

# Define the locations
emp_location = 'spark-warehouse/hive_emp'
dept_location = 'spark-warehouse/hive_dept'

# Remove the existing directories if they exist
if os.path.exists(emp_location):
    rmtree(emp_location)
if os.path.exists(dept_location):
    rmtree(dept_location)


In [58]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Create DataFrames") \
    .enableHiveSupport() \
    .getOrCreate()

# Define the schema for the emp DataFrame
emp_schema = StructType([
    StructField("emp_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("dept_id", StringType(), True),
    StructField("salary", IntegerType(), True)
])

# Define the schema for the dept DataFrame
dept_schema = StructType([
    StructField("name", StringType(), True),
    StructField("dept_id", StringType(), True)
])

# Create a DataFrame for employees
emp = [(1, "Smith", "fi", 1000),
       (2, "Rose", "ma", 2000),
       (3, "Williams", "ma", 7000),
       (4, "Jones", "sa", 2000),
       (5, "Brown", "sa", 1000),
       (6, "Katie", "fi", 3500),
       (7, "Linda", "it", 2000),
       (8, "Michael", "it", 2000),
       (9, "Johnson", "ma", 1000),
       (10, "Tom", "fi", 4300),
       (11, "John", "fi", 10000),
       (12, "Doe", "ma", 5000),
       (13, "Ben", "sa", 3000),
       (14, "Justin", "it", 5000),
       (15, "Lily", "cs", 100),
       (None, None, None, 1000),
       (17, "Jane", None, 9000),
       (18, "Lisa", None, 2500),
       (19, "Lucy", None, 5000),
       (20, "Kate", None, 3000)]

# Create a DataFrame for departments
dept = [("Finance", "fi"),
        ("Marketing", "ma"),
        ("Sales", "sa"),
        ("Computer Science", "cs"),
        ("Info Tech", "it")]

# Create the DataFrames
df = spark.createDataFrame(emp, emp_schema)
deptdf = spark.createDataFrame(dept, dept_schema)

# Create Temp Tables for SQL
df.createOrReplaceTempView("empdf")
deptdf.createOrReplaceTempView("deptdf")

# Drop the existing tables if they exist
spark.sql("DROP TABLE IF EXISTS hive_emp")
spark.sql("DROP TABLE IF EXISTS hive_dept")

# Save as Hive table
df.write.saveAsTable("hive_emp", mode="overwrite")
deptdf.write.saveAsTable("hive_dept", mode="overwrite")

# Show the DataFrames to verify
df.show()
deptdf.show()


+------+--------+-------+------+
|emp_id|    name|dept_id|salary|
+------+--------+-------+------+
|     1|   Smith|     fi|  1000|
|     2|    Rose|     ma|  2000|
|     3|Williams|     ma|  7000|
|     4|   Jones|     sa|  2000|
|     5|   Brown|     sa|  1000|
|     6|   Katie|     fi|  3500|
|     7|   Linda|     it|  2000|
|     8| Michael|     it|  2000|
|     9| Johnson|     ma|  1000|
|    10|     Tom|     fi|  4300|
|    11|    John|     fi| 10000|
|    12|     Doe|     ma|  5000|
|    13|     Ben|     sa|  3000|
|    14|  Justin|     it|  5000|
|    15|    Lily|     cs|   100|
|  NULL|    NULL|   NULL|  1000|
|    17|    Jane|   NULL|  9000|
|    18|    Lisa|   NULL|  2500|
|    19|    Lucy|   NULL|  5000|
|    20|    Kate|   NULL|  3000|
+------+--------+-------+------+

+----------------+-------+
|            name|dept_id|
+----------------+-------+
|         Finance|     fi|
|       Marketing|     ma|
|           Sales|     sa|
|Computer Science|     cs|
|       Info Tech|

## Broadcast Join
Size of broadcast table is 10MB.
Can change threshold up to 8GB

In [59]:
# Check size of transmission table
threshold_str = spark.conf.get("spark.sql.autoBroadcastJoinThreshold")
threshold_value = int(threshold_str.rstrip('b'))
size = threshold_value / (1024 * 1024)
print(f"Default size of broadcast table is {size} MB.")

Default size of broadcast table is 50.0 MB.


In [60]:
# set size of streaming table as 50mb
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 50 * 1024 * 1024)

In [61]:
join_df = df.join(broadcast(deptdf), df["dept_id"] == deptdf["dept_id"])

## Caching
Use cache/persistence function to keep the dataframe in memoery

In [62]:
# Cache the DataFrame
df.cache()

# Trigger an action to ensure caching is materialized
df.count()

# Print the storage levels
print("Memory used: {0}".format(df.storageLevel.useMemory))
print("Disk used: {0}".format(df.storageLevel.useDisk))

# You can cache again and re-check the storage level if necessary, but it's not needed
# df.cache()
# df.count()
# Print the storage levels again if you want to check after caching again
print("Memory used after re-cache: {0}".format(df.storageLevel.useMemory))
print("Disk used after re-cache: {0}".format(df.storageLevel.useDisk))

Memory used: True
Disk used: True
Memory used after re-cache: True
Disk used after re-cache: True


When use the `cache()` function, it will use storage tier as Memory_Only (~2.0.2) and Memory_and_DISK (2.1.x afterwards) 
But we can use `persist()` to specify levels of storage 

In [63]:
from pyspark.storagelevel import StorageLevel


In [64]:
deptdf.persist(StorageLevel.MEMORY_ONLY)
deptdf.count()
print("Memory used for deptdf: {0}".format(deptdf.storageLevel.useMemory))
print("Disk used for deptdf: {0}".format(deptdf.storageLevel.useDisk))

Memory used for deptdf: True
Disk used for deptdf: False


## Dont persist
Clear cache of data when no longer needed!

In [65]:
df.unpersist()

DataFrame[emp_id: int, name: string, dept_id: string, salary: int]

In [66]:
deptdf.unpersist()

DataFrame[name: string, dept_id: string]

In [67]:
# clear cache
spark.catalog.clearCache()

## SQL Expressions

In [68]:
from pyspark.sql.functions import expr

script = """case when salary > 5000 then 'high'
                when salary > 3000 then 'medium'
                when salary > 1000 then 'low'
                else 'invalid'
        end as salary_level"""

newdf =   df.withColumn("salary_level", expr(script))
newdf.show()

+------+--------+-------+------+------------+
|emp_id|    name|dept_id|salary|salary_level|
+------+--------+-------+------+------------+
|     1|   Smith|     fi|  1000|     invalid|
|     2|    Rose|     ma|  2000|         low|
|     3|Williams|     ma|  7000|        high|
|     4|   Jones|     sa|  2000|         low|
|     5|   Brown|     sa|  1000|     invalid|
|     6|   Katie|     fi|  3500|      medium|
|     7|   Linda|     it|  2000|         low|
|     8| Michael|     it|  2000|         low|
|     9| Johnson|     ma|  1000|     invalid|
|    10|     Tom|     fi|  4300|      medium|
|    11|    John|     fi| 10000|        high|
|    12|     Doe|     ma|  5000|      medium|
|    13|     Ben|     sa|  3000|         low|
|    14|  Justin|     it|  5000|      medium|
|    15|    Lily|     cs|   100|     invalid|
|  NULL|    NULL|   NULL|  1000|     invalid|
|    17|    Jane|   NULL|  9000|        high|
|    18|    Lisa|   NULL|  2500|         low|
|    19|    Lucy|   NULL|  5000|  

## Using the selectExpr func

In [69]:
newdf = df.selectExpr("*", script)
newdf.show()

+------+--------+-------+------+------------+
|emp_id|    name|dept_id|salary|salary_level|
+------+--------+-------+------+------------+
|     1|   Smith|     fi|  1000|     invalid|
|     2|    Rose|     ma|  2000|         low|
|     3|Williams|     ma|  7000|        high|
|     4|   Jones|     sa|  2000|         low|
|     5|   Brown|     sa|  1000|     invalid|
|     6|   Katie|     fi|  3500|      medium|
|     7|   Linda|     it|  2000|         low|
|     8| Michael|     it|  2000|         low|
|     9| Johnson|     ma|  1000|     invalid|
|    10|     Tom|     fi|  4300|      medium|
|    11|    John|     fi| 10000|        high|
|    12|     Doe|     ma|  5000|      medium|
|    13|     Ben|     sa|  3000|         low|
|    14|  Justin|     it|  5000|      medium|
|    15|    Lily|     cs|   100|     invalid|
|  NULL|    NULL|   NULL|  1000|     invalid|
|    17|    Jane|   NULL|  9000|        high|
|    18|    Lisa|   NULL|  2500|         low|
|    19|    Lucy|   NULL|  5000|  

## User Defined Functions (UDFs)

### Python function to find salary_level for a given salary

In [70]:
def salary_level(sal):
    level = None
    if sal > 5000:
        level = 'high'
    elif sal > 3000:
        level = 'medium'
    elif sal > 1000:
        level = 'low'
    else:
        level = 'invalid'
    return level
    

In [71]:
sal_level = udf(salary_level, StringType()) 


In [72]:
newdf = df.withColumn("salary_level", sal_level(df.salary))
newdf.show()

+------+--------+-------+------+------------+
|emp_id|    name|dept_id|salary|salary_level|
+------+--------+-------+------+------------+
|     1|   Smith|     fi|  1000|     invalid|
|     2|    Rose|     ma|  2000|         low|
|     3|Williams|     ma|  7000|        high|
|     4|   Jones|     sa|  2000|         low|
|     5|   Brown|     sa|  1000|     invalid|
|     6|   Katie|     fi|  3500|      medium|
|     7|   Linda|     it|  2000|         low|
|     8| Michael|     it|  2000|         low|
|     9| Johnson|     ma|  1000|     invalid|
|    10|     Tom|     fi|  4300|      medium|
|    11|    John|     fi| 10000|        high|
|    12|     Doe|     ma|  5000|      medium|
|    13|     Ben|     sa|  3000|         low|
|    14|  Justin|     it|  5000|      medium|
|    15|    Lily|     cs|   100|     invalid|
|  NULL|    NULL|   NULL|  1000|     invalid|
|    17|    Jane|   NULL|  9000|        high|
|    18|    Lisa|   NULL|  2500|         low|
|    19|    Lucy|   NULL|  5000|  

### Work with NULL values

In [73]:
# isNull()
newdf = df.filter(df.salary.isNull())
newdf.count()

0

In [74]:
# isNotNull()
df.filter(df.salary.isNotNull()).count()

20

In [75]:
df.printSchema()

root
 |-- emp_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- salary: integer (nullable = true)



In [76]:
# fillna()
newdf = df.fillna("INVALID", ["dept_id"])
newdf.show()

+------+--------+-------+------+
|emp_id|    name|dept_id|salary|
+------+--------+-------+------+
|     1|   Smith|     fi|  1000|
|     2|    Rose|     ma|  2000|
|     3|Williams|     ma|  7000|
|     4|   Jones|     sa|  2000|
|     5|   Brown|     sa|  1000|
|     6|   Katie|     fi|  3500|
|     7|   Linda|     it|  2000|
|     8| Michael|     it|  2000|
|     9| Johnson|     ma|  1000|
|    10|     Tom|     fi|  4300|
|    11|    John|     fi| 10000|
|    12|     Doe|     ma|  5000|
|    13|     Ben|     sa|  3000|
|    14|  Justin|     it|  5000|
|    15|    Lily|     cs|   100|
|  NULL|    NULL|INVALID|  1000|
|    17|    Jane|INVALID|  9000|
|    18|    Lisa|INVALID|  2500|
|    19|    Lucy|INVALID|  5000|
|    20|    Kate|INVALID|  3000|
+------+--------+-------+------+



In [77]:
# dropna()
newdf = df.dropna()
newdf.show()

+------+--------+-------+------+
|emp_id|    name|dept_id|salary|
+------+--------+-------+------+
|     1|   Smith|     fi|  1000|
|     2|    Rose|     ma|  2000|
|     3|Williams|     ma|  7000|
|     4|   Jones|     sa|  2000|
|     5|   Brown|     sa|  1000|
|     6|   Katie|     fi|  3500|
|     7|   Linda|     it|  2000|
|     8| Michael|     it|  2000|
|     9| Johnson|     ma|  1000|
|    10|     Tom|     fi|  4300|
|    11|    John|     fi| 10000|
|    12|     Doe|     ma|  5000|
|    13|     Ben|     sa|  3000|
|    14|  Justin|     it|  5000|
|    15|    Lily|     cs|   100|
+------+--------+-------+------+



In [78]:
newdf = df.dropna(how='all') # if all values are NA drop that row or column
newdf.show()

+------+--------+-------+------+
|emp_id|    name|dept_id|salary|
+------+--------+-------+------+
|     1|   Smith|     fi|  1000|
|     2|    Rose|     ma|  2000|
|     3|Williams|     ma|  7000|
|     4|   Jones|     sa|  2000|
|     5|   Brown|     sa|  1000|
|     6|   Katie|     fi|  3500|
|     7|   Linda|     it|  2000|
|     8| Michael|     it|  2000|
|     9| Johnson|     ma|  1000|
|    10|     Tom|     fi|  4300|
|    11|    John|     fi| 10000|
|    12|     Doe|     ma|  5000|
|    13|     Ben|     sa|  3000|
|    14|  Justin|     it|  5000|
|    15|    Lily|     cs|   100|
|  NULL|    NULL|   NULL|  1000|
|    17|    Jane|   NULL|  9000|
|    18|    Lisa|   NULL|  2500|
|    19|    Lucy|   NULL|  5000|
|    20|    Kate|   NULL|  3000|
+------+--------+-------+------+



In [79]:
# remove all reows where dept_id is null
newdf = df.dropna(subset="dept_id")
newdf.show()

+------+--------+-------+------+
|emp_id|    name|dept_id|salary|
+------+--------+-------+------+
|     1|   Smith|     fi|  1000|
|     2|    Rose|     ma|  2000|
|     3|Williams|     ma|  7000|
|     4|   Jones|     sa|  2000|
|     5|   Brown|     sa|  1000|
|     6|   Katie|     fi|  3500|
|     7|   Linda|     it|  2000|
|     8| Michael|     it|  2000|
|     9| Johnson|     ma|  1000|
|    10|     Tom|     fi|  4300|
|    11|    John|     fi| 10000|
|    12|     Doe|     ma|  5000|
|    13|     Ben|     sa|  3000|
|    14|  Justin|     it|  5000|
|    15|    Lily|     cs|   100|
+------+--------+-------+------+



### partitioning

In [85]:
df.rdd.getNumPartitions()

12

In [81]:
df.show()

+------+--------+-------+------+
|emp_id|    name|dept_id|salary|
+------+--------+-------+------+
|     1|   Smith|     fi|  1000|
|     2|    Rose|     ma|  2000|
|     3|Williams|     ma|  7000|
|     4|   Jones|     sa|  2000|
|     5|   Brown|     sa|  1000|
|     6|   Katie|     fi|  3500|
|     7|   Linda|     it|  2000|
|     8| Michael|     it|  2000|
|     9| Johnson|     ma|  1000|
|    10|     Tom|     fi|  4300|
|    11|    John|     fi| 10000|
|    12|     Doe|     ma|  5000|
|    13|     Ben|     sa|  3000|
|    14|  Justin|     it|  5000|
|    15|    Lily|     cs|   100|
|  NULL|    NULL|   NULL|  1000|
|    17|    Jane|   NULL|  9000|
|    18|    Lisa|   NULL|  2500|
|    19|    Lucy|   NULL|  5000|
|    20|    Kate|   NULL|  3000|
+------+--------+-------+------+



### repartition

In [87]:
newdf = df.repartition(15)
newdf.rdd.getNumPartitions()

15

**this is expensive operation as it requires DATA SHUFFLING between workers**

### coalesce (only decrease nums of partition)

In [88]:
newdf = df.coalesce(5)
newdf.rdd.getNumPartitions()

5

- By default, number of parittions for Spark SQL is 200
- Can also set num of parttitions at the Spark application level

In [89]:
# set num of partitions as Spark Application
spark.conf.set("spark.sql.shuffle.partitions", "500")

num_part = spark.conf.get("spark.sql.shuffle.partitions")
print("Number of Partitions: {0}".format(num_part))

Number of Partitions: 500


## API Catalog
This is a user-facing API, which can access via SparkSession.catalog

- **listDatabases()**: return all databases along with their location on file system

In [90]:
spark.catalog.listDatabases()

[Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/F:/Data_Engineering/Apache_Spark/BigDataSpark/3_Advanced_Features/spark-warehouse')]

- **listTables()**: return all tables for a given database along with info such as the table type (foreign/managed) and whether a partitcular table is temporary or permanent -> this includes all temp views

In [91]:
spark.catalog.listTables("default")

[Table(name='hive_dept', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='hive_emp', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='deptdf', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='empdf', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

- **listColumns()**: return all columns from a particular table in Database (return data types, whether column used in partitions or pools)

In [93]:
spark.catalog.listColumns("hive_emp", "default")

D:\spark\python\pyspark\sql\catalog.py:651: FutureWarning: `dbName` has been deprecated since Spark 3.4 and might be removed in a future version. Use listColumns(`dbName.tableName`) instead.
  warnings.warn(


[Column(name='emp_id', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='name', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='dept_id', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='salary', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False)]

- **listFunctions()**: return all features available in SparkSession along with the info it is temporary or not

In [94]:
spark.catalog.listFunctions()

[Function(name='!', catalog=None, namespace=None, description='! expr - Logical not.', className='org.apache.spark.sql.catalyst.expressions.Not', isTemporary=True),
 Function(name='!=', catalog=None, namespace=None, description='expr1 != expr2 - Returns true if `expr1` is not equal to `expr2`.', className=None, isTemporary=True),
 Function(name='%', catalog=None, namespace=None, description='expr1 % expr2 - Returns the remainder after `expr1`/`expr2`.', className='org.apache.spark.sql.catalyst.expressions.Remainder', isTemporary=True),
 Function(name='&', catalog=None, namespace=None, description='expr1 & expr2 - Returns the result of bitwise AND of `expr1` and `expr2`.', className='org.apache.spark.sql.catalyst.expressions.BitwiseAnd', isTemporary=True),
 Function(name='*', catalog=None, namespace=None, description='expr1 * expr2 - Returns `expr1`*`expr2`.', className='org.apache.spark.sql.catalyst.expressions.Multiply', isTemporary=True),
 Function(name='+', catalog=None, namespace=N

- **currentDatabase()**

In [95]:
spark.catalog.currentDatabase()

'default'

- **setCurrentDatabase()**

In [ ]:
spark.catalog.setCurrentDatabase("company")

- **cacheTable()**

In [97]:
spark.catalog.cacheTable("default.hive_emp")

- **isCached()**

In [98]:
spark.catalog.isCached("default.hive_emp")

True

- **uncacheTable()**

In [99]:
spark.catalog.uncacheTable("default.hive_emp")

In [100]:
# Verify if the table is uncached
spark.catalog.isCached("default.hive_emp")

False

In [101]:
# Clear cache
spark.catalog.clearCache()